In [1]:
# List all device
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [2]:
# Check available GPU
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [4]:
# Importing the libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Reshape, Lambda
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.activations import softmax
from keras.optimizers import SGD
import math
import pickle

In [5]:
with open("../data/dowjones_calculated/absolute_periods750_250_240.txt", "rb") as fp:   # Unpickling
    dataset = pickle.load(fp)

In [6]:
def normalize_data(df):
    """normalize a dataframe."""
    mean = df.mean(axis=1)
    std = df.std(axis=1)
    df = df.sub(mean, axis=0)
    df = df.div(std, axis=0)
    df = df.values
    return df

In [7]:
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

In [8]:
i = 0
x_train1 = normalize_data(dataset[0][i][0])
y_train1 = get_one_hot(dataset[0][i][1].values, 2) * 1.0
x_test1 = normalize_data(dataset[1][i][0])
y_test1 = get_one_hot(dataset[1][i][1].values, 2) * 1.0

In [9]:
print(f"x train 1 shape: {x_train1.shape}")
print(f"y train 1 shape: {y_train1.shape}")
print(f"x test 1 shape: {x_test1.shape}")
print(f"y test 1 shape: {y_test1.shape}")

x train 1 shape: (750, 31)
y train 1 shape: (750, 31, 2)
x test 1 shape: (490, 31)
y test 1 shape: (490, 31, 2)


In [10]:
x_series = [x_train1[i:i+240,j] for j in range(31) for i in range(750 - 240)]
y_series = [y_train1[i+240,j] for j in range(31) for i in range(750 - 240)]
x = np.array(x_series)
y = np.array(y_series)
print(f"x shape: {x.shape}")
print(f"y shape: {y.shape}")

x shape: (15810, 240)
y shape: (15810, 2)


In [11]:
# x = x.transpose((0,2,1))
# x = np.reshape(x, (510 * 31, timestep))
# y = np.reshape(y, (510 * 31, 2))
print(f"x shape: {x.shape}")
print(f"y shape: {y.shape}")

x shape: (15810, 240)
y shape: (15810, 2)


In [12]:
x = np.reshape(x, (x.shape[0], x.shape[1], 1))
print(f"x shape: {x.shape}")


x shape: (15810, 240, 1)


In [13]:
i = 0
timestep = 240
x_train = normalize_data(dataset[0][i][0])
y_train = get_one_hot(dataset[0][i][1].values, 2) * 1.0
x_test = normalize_data(dataset[1][i][0])
y_test = get_one_hot(dataset[1][i][1].values, 2) * 1.0

train_gen = TimeseriesGenerator(x_train, y_train,
                                length=timestep, sampling_rate=1,
                                batch_size=510)
test_gen = TimeseriesGenerator(x_test, y_test,
                               length=timestep, sampling_rate=1,
                               batch_size=250)

In [14]:
print(f"x train shape: {x_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {y_test.shape}")
print(f"y test shape: {y_test.shape}")
# print(f"predicted shape: {predicted.shape}")

x train shape: (750, 31)
y train shape: (750, 31, 2)
x test shape: (490, 31, 2)
y test shape: (490, 31, 2)


In [15]:
x_train = train_gen[0][0]
y_train = train_gen[0][1]
y_test = test_gen[0][0]
y_test = test_gen[0][1]

In [16]:
print(f"x train shape: {x_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {y_test.shape}")
print(f"y test shape: {y_test.shape}")

x train shape: (510, 240, 31)
y train shape: (510, 31, 2)
x test shape: (250, 31, 2)
y test shape: (250, 31, 2)


In [17]:
# Reshaping X_train for efficient modelling
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))

In [18]:
print(f"x train shape: {x_train.shape}")
print(f"y train shape: {y_train.shape}")
print(f"x test shape: {y_test.shape}")
print(f"y test shape: {y_test.shape}")

x train shape: (510, 240, 31)
y train shape: (510, 31, 2)
x test shape: (250, 31, 2)
y test shape: (250, 31, 2)


In [19]:
# expected input data shape: (batch_size, timesteps, data_dim)
regressor = Sequential()
regressor.add(LSTM(units=250,
                   input_shape=(timestep, 1), dropout=0.1))
# regressor.add(LSTM(units=25, return_sequences=True,
#                    input_shape=(timestep, 1), dropout=0.1))
# regressor.add(LSTM(25, return_sequences=True, dropout=0.1))
# regressor.add(LSTM(25, return_sequences=True, dropout=0.1))
# regressor.add(LSTM(25, dropout=0.1))

# regressor.add(Dense(10,input_shape=(timestep, ), activation='relu'))
regressor.add(Dense(100, activation='relu'))
regressor.add(Dense(100, activation='relu'))

# regressor.add(Reshape((31, 2)))
# regressor.add(Lambda(lambda x: softmax(x, axis=-1)))

regressor.add(Dense(2, activation='softmax'))

regressor.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

regressor.fit(x, y, epochs=1000,batch_size=1000, validation_split=0.2, callbacks = [EarlyStopping(monitor='val_loss', mode='min', patience=100),
             ModelCheckpoint(filepath='../model/LSTM/best_model.h5', monitor='val_acc', save_best_only=True)])

# regressor.fit_generator(train_gen, steps_per_epoch=len(train_gen),
#                         epochs=10, validation_data=test_gen,
#                         callbacks=[
#                             EarlyStopping(monitor='val_loss',
#                                           mode='min', patience=10),
#                             ModelCheckpoint(filepath="../model/mymodel.h5",
#                                             monitor='val_acc',
#                                             save_best_only=True)])


Train on 12648 samples, validate on 3162 samples
Epoch 1/1000
12648/12648 [==============================] - 12s 966us/step - loss: 0.6937 - acc: 0.5023 - val_loss: 0.6931 - val_acc: 0.5130
Epoch 2/1000
12648/12648 [==============================] - 11s 884us/step - loss: 0.6932 - acc: 0.5071 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 3/1000
12648/12648 [==============================] - 11s 885us/step - loss: 0.6930 - acc: 0.5087 - val_loss: 0.6930 - val_acc: 0.5070
Epoch 4/1000
12648/12648 [==============================] - 11s 879us/step - loss: 0.6932 - acc: 0.5090 - val_loss: 0.6932 - val_acc: 0.4991
Epoch 5/1000
12648/12648 [==============================] - 11s 879us/step - loss: 0.6931 - acc: 0.5081 - val_loss: 0.6932 - val_acc: 0.5060
Epoch 6/1000
12648/12648 [==============================] - 13s 989us/step - loss: 0.6928 - acc: 0.5155 - val_loss: 0.6937 - val_acc: 0.5000
Epoch 7/1000
12648/12648 [==============================] - 11s 877us/step - loss: 0.6930 - acc: 0.5051 -

KeyboardInterrupt: 